In [11]:
"""
Notebook : Setup et test du Vector Store FAISS
Objectif : Créer l'index vectoriel pour accélérer le matching
"""

# ========================================
# CELLULE 1 : Imports et setup
# ========================================

import sys
sys.path.append('../src')

import json
from pathlib import Path
from vector_store import JobVectorStore, create_vector_store_from_dataset, load_vector_store
import time

print("✅ Imports terminés")

✅ Imports terminés


In [12]:
# ========================================
# CELLULE 2 : Charger les offres d'emploi
# ========================================

jobs_path = Path("../data/jobs/jobs_dataset.json")

if not jobs_path.exists():
    print("❌ Fichier jobs_dataset.json introuvable")
    print("💡 Exécutez d'abord 04_job_generation.ipynb")
else:
    with open(jobs_path, 'r', encoding='utf-8') as f:
        jobs_data = json.load(f)
    
    # Vérifier la structure du fichier
    if isinstance(jobs_data, dict):
        # Si c'est un dictionnaire (organisé par catégorie)
        print(f"✅ Jobs dataset chargé (structure par catégorie)")
        print(f"📂 Catégories : {list(jobs_data.keys())}")
        
        # Aplatir en une seule liste
        jobs = []
        for category, job_list in jobs_data.items():
            if isinstance(job_list, list):
                jobs.extend(job_list)
        
        print(f"✅ {len(jobs)} offres chargées au total\n")
        
    elif isinstance(jobs_data, list):
        # Si c'est déjà une liste
        jobs = jobs_data
        print(f"✅ {len(jobs)} offres chargées\n")
    else:
        print("❌ Format de fichier JSON non reconnu")
        jobs = []
    
    # Afficher un exemple
    if jobs:
        print(f"📋 Exemple d'offre :")
        print(json.dumps(jobs[0], indent=2, ensure_ascii=False))
    else:
        print("⚠️ Aucune offre trouvée dans le fichier")

✅ Jobs dataset chargé (structure par catégorie)
📂 Catégories : ['metadata', 'jobs']
✅ 25 offres chargées au total

📋 Exemple d'offre :
{
  "job_id": "job_001",
  "category": "ml_engineer",
  "title": "Junior ML Engineer",
  "company": "AI Startup Paris",
  "location": "Toulouse, France",
  "type": "CDI",
  "experience": "0-2 ans",
  "salary": "35-45K€",
  "description": "Nous recherchons un Junior ML Engineer passionné pour rejoindre notre équipe R&D.\n\nResponsabilités :\n- Développer des modèles de Machine Learning (classification, régression)\n- Entraîner et optimiser des réseaux de neurones avec PyTorch\n- Déployer des modèles en production avec Docker\n- Participer aux revues de code et à l'amélioration continue\n\nStack technique :\n- Python, PyTorch, scikit-learn\n- Docker, Git\n- FastAPI pour les APIs\n- PostgreSQL",
  "requirements": [
    "Python (numpy, pandas, scikit-learn)",
    "Machine Learning basics (supervised learning)",
    "Git et GitHub",
    "Docker (notions de b

le code Lit le fichier jobs_dataset.json. Si les offres sont organisées par catégorie (dict), elle les fusionne en une liste unique. 

Les offres chargées ici seront indexées par FAISS dans la Cellule 3  /// FAISS est la  bibliothèque développée par Facebook (Meta)  pour la recherche de similarité ultra-rapide dans des vecteurs de grande dimension 

In [13]:
# ========================================
# CELLULE 3 : Créer et entraîner le Vector Store (option A avec all-mpnet-base-v2	Dimensions : 768)
# ========================================

print("🔨 Création du Vector Store FAISS...")
print("⏱️ Temps estimé : 30-60 secondes\n")

start_time = time.time()

# Créer vector store
vector_store = JobVectorStore(model_name="all-mpnet-base-v2")

# Construire index
vector_store.build_index(jobs)

elapsed = time.time() - start_time

print(f"\n✅ Index créé en {elapsed:.1f} secondes")
print(f"📊 Statistiques : {vector_store.get_stats()}")

🔨 Création du Vector Store FAISS...
⏱️ Temps estimé : 30-60 secondes



Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


🔨 Construction de l'index FAISS pour 25 offres...
📊 Génération des embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🔧 Création de l'index FAISS...
✅ Index construit : 25 offres indexées

✅ Index créé en 14.7 secondes
📊 Statistiques : {'status': 'trained', 'total_jobs': 25, 'dimension': 768, 'model': 'all-mpnet-base-v2'}


- Charge le modèle Sentence-Transformers (all-mpnet-base-v2) pour créer des embeddings
- Pour chaque offre :
** Combine titre + description → "Data Scientist. Python, ML, TensorFlow..."
** Transforme ce texte en vecteur de 768 dimensions (embedding)

- Crée un index FAISS qui stocke ces 25 vecteurs de manière optimisée
- Normalise les vecteurs pour calculer la similarité cosinus

In [14]:
# ========================================
# CELLULE 4 : Test de recherche (CV exemple)
# ========================================

# Charger un CV de test
cv_text_path = Path("cv_text_pdfplumber.txt")

if cv_text_path.exists():
    with open(cv_text_path, 'r', encoding='utf-8') as f:
        cv_text = f.read()
    
    print("✅ CV de test chargé\n")
    
    # Recherche rapide
    print("🔍 Recherche des 10 offres les plus pertinentes...")
    
    start_search = time.time()
    results = vector_store.search(cv_text, top_k=10, min_score=0.3)
    search_time = time.time() - start_search
    
    print(f"⚡ Recherche terminée en {search_time:.3f} secondes\n")
    
    # Afficher résultats
    print(f"🎯 TOP {len(results)} RÉSULTATS :\n")
    for i, job in enumerate(results, 1):
        print(f"{i}. {job['title']} - {job['company']}")
        print(f"   Score FAISS : {job['faiss_score_percent']:.1f}%")
        print(f"   Localisation : {job['location']}")
        print()

else:
    print("⚠️ Pas de CV de test trouvé")
    print("💡 Utilisez 01_cv_parser.ipynb pour générer cv_text_pdfplumber.txt")

✅ CV de test chargé

🔍 Recherche des 10 offres les plus pertinentes...
⚡ Recherche terminée en 3.192 secondes

🎯 TOP 10 RÉSULTATS :

1. Junior ML Engineer - AI Startup Paris
   Score FAISS : 41.0%
   Localisation : Toulouse, France

2. Data Scientist - Finance - FinTech Solutions
   Score FAISS : 38.2%
   Localisation : Toulouse, France

3. Senior ML Engineer - BigTech France
   Score FAISS : 37.9%
   Localisation : Remote France

4. SRE Engineer - ScaleOps
   Score FAISS : 37.6%
   Localisation : Paris, France

5. MLOps Engineer - DataCorp
   Score FAISS : 35.8%
   Localisation : Lyon, France

6. ML Engineer - Computer Vision - VisionTech
   Score FAISS : 35.6%
   Localisation : Paris (Hybrid)

7. Senior Python Developer - Enterprise Tech
   Score FAISS : 35.3%
   Localisation : Paris (Hybrid)

8. Python Developer - Data Engineering - BigData Corp
   Score FAISS : 33.3%
   Localisation : Bordeaux, France

9. Full Stack JavaScript Developer - Digital Agency
   Score FAISS : 33.2%
   Lo

- Transforme ton CV en vecteur (embedding)
- Compare ce vecteur avec les 25 offres stockées dans FAISS
- Retourne les 10 offres les plus similaires avec un score de 0 à 100%

Pourquoi c'est rapide ?

* Sans FAISS : Il faudrait calculer 25 embeddings à chaque recherche (~30s)
* Avec FAISS : Les embeddings sont déjà calculés, on fait juste une comparaison (~0.5s)

cette cellule Utilise l'index de la Cellule 3 pour chercher. Les résultats seront comparés dans la Cellule 7.

In [15]:
# ========================================
# CELLULE 5 : Sauvegarder l'index
# ========================================

# Créer dossier models/
models_dir = Path("../models")
models_dir.mkdir(exist_ok=True)

# Sauvegarder
index_path = models_dir / "faiss_jobs.index"
metadata_path = models_dir / "faiss_jobs_metadata.pkl"

vector_store.save(str(index_path), str(metadata_path))

print(f"\n✅ Vector Store sauvegardé !")
print(f"📁 Index : {index_path}")
print(f"📁 Métadonnées : {metadata_path}")
print(f"💾 Taille index : {index_path.stat().st_size / 1024:.1f} KB")

✅ Index sauvegardé : ../models/faiss_jobs.index
✅ Métadonnées sauvegardées : ../models/faiss_jobs_metadata.pkl
📌 Modèle utilisé : all-mpnet-base-v2 (768 dimensions)

✅ Vector Store sauvegardé !
📁 Index : ../models/faiss_jobs.index
📁 Métadonnées : ../models/faiss_jobs_metadata.pkl
💾 Taille index : 75.0 KB


Ce qu'elle fait :

* Sauvegarde l'index FAISS sur le disque (fichier .index)
* Sauvegarde les métadonnées (titres, entreprises, etc.) dans un fichier .pkl

Pourquoi c'est important ?

* Sans sauvegarde : Tu dois recalculer les embeddings à chaque fois (30-60s)
* Avec sauvegarde : Tu charges l'index en 2 secondes et c'est prêt !


Ces fichiers seront rechargés dans la Cellule 6 et utilisés par l'API plus tard.

In [16]:
# ========================================
# CELLULE 6 : Test de chargement (AUTO-DÉTECTION)
# ========================================

print("🔄 Test de rechargement de l'index...\n")

# 🆕 Plus besoin de spécifier le modèle !
# Il sera automatiquement détecté depuis les métadonnées
vs_loaded = JobVectorStore()

# Charger index sauvegardé
vs_loaded.load(str(index_path), str(metadata_path))

print(f"✅ Index rechargé : {vs_loaded.get_stats()}")

# Test recherche rapide
if cv_text_path.exists():
    start = time.time()
    results_loaded = vs_loaded.search(cv_text, top_k=5)
    elapsed = time.time() - start
    
    print(f"\n⚡ Recherche avec index chargé : {elapsed:.3f}s")
    print(f"🎯 {len(results_loaded)} résultats retournés")
else:
    print("⚠️ Pas de CV pour tester")

🔄 Test de rechargement de l'index...



Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Index chargé : 25 offres
📌 Modèle final : all-mpnet-base-v2 (768 dimensions)
✅ Index rechargé : {'status': 'trained', 'total_jobs': 25, 'dimension': 768, 'model': 'all-mpnet-base-v2'}

⚡ Recherche avec index chargé : 2.046s
🎯 5 résultats retournés


Ce qu'elle fait :

    Crée un nouveau vector store (vide)
    Charge l'index sauvegardé dans la Cellule 5
    Teste une recherche pour vérifier que tout fonctionne

Pourquoi ce test ?

    Vérifie que la sauvegarde a bien marché
    Simule ce qui se passera dans l'API (charger un index existant)

In [17]:
# ========================================
# CELLULE 7 : Comparaison de performances
# ========================================

print("\n" + "="*60)
print("📊 COMPARAISON DE PERFORMANCES")
print("="*60)

if cv_text_path.exists():
    # Méthode 1 : Sans FAISS (matching classique - simulation)
    print("\n🐢 Méthode classique (sans FAISS)")
    print("   Temps estimé : ~30-60 secondes")
    print("   Raison : Calcul embeddings + comparaison pour chaque offre")
    
    # Méthode 2 : Avec FAISS
    print("\n⚡ Méthode FAISS (avec index)")
    start = time.time()
    faiss_results = vector_store.search(cv_text, top_k=25)
    faiss_time = time.time() - start
    
    print(f"   Temps réel : {faiss_time:.3f} secondes")
    print(f"   Gain : ~{60/faiss_time:.0f}x plus rapide !")
    
    print("\n✅ FAISS est LA solution pour scaler à 100+ offres")

else:
    print("\n⚠️ Pas de CV pour tester les performances")


📊 COMPARAISON DE PERFORMANCES

🐢 Méthode classique (sans FAISS)
   Temps estimé : ~30-60 secondes
   Raison : Calcul embeddings + comparaison pour chaque offre

⚡ Méthode FAISS (avec index)
   Temps réel : 3.455 secondes
   Gain : ~17x plus rapide !

✅ FAISS est LA solution pour scaler à 100+ offres


Ce qu'elle fait :

    Mesure le temps de recherche avec FAISS
    Compare avec le temps sans FAISS (estimé à 30-60s)
    Calcule le gain de performance

In [18]:
# ========================================
# CELLULE 8 : VALIDATION FINALE
# ========================================

print("\n" + "="*60)
print(" VALIDATION - VECTOR STORE FAISS")
print("="*60)

checks = {
    "✅ Index FAISS créé": vector_store.is_trained,
    "✅ Index sauvegardé sur disque": index_path.exists(),
    "✅ Métadonnées sauvegardées": metadata_path.exists(),
    "✅ Rechargement fonctionne": vs_loaded.is_trained,
    "✅ Recherche rapide (<1s)": 'faiss_time' in locals() and faiss_time < 1.0,
    "✅ Résultats pertinents": len(results) > 0 if 'results' in locals() else False
}

for check, passed in checks.items():
    status = check if passed else check.replace("✅", "❌")
    print(status)

score = sum(checks.values())
total = len(checks)

print(f"\n📊 SCORE : {score}/{total} ({score/total*100:.0f}%)")

if score == total:
    print("\n🎉 VECTOR STORE OPÉRATIONNEL !")
    print("👉 Prochaine étape : Intégrer FAISS dans l'API")
else:
    print("\n⚠️ Quelques ajustements nécessaires")

print("\n💡 Utilisation dans l'API :")
print("```python")
print("from vector_store import load_vector_store")
print("vs = load_vector_store()")
print("results = vs.search(cv_text, top_k=10)")
print("```")


 VALIDATION - VECTOR STORE FAISS
✅ Index FAISS créé
✅ Index sauvegardé sur disque
✅ Métadonnées sauvegardées
✅ Rechargement fonctionne
❌ Recherche rapide (<1s)
✅ Résultats pertinents

📊 SCORE : 5/6 (83%)

⚠️ Quelques ajustements nécessaires

💡 Utilisation dans l'API :
```python
from vector_store import load_vector_store
vs = load_vector_store()
results = vs.search(cv_text, top_k=10)
```


In [19]:
# ========================================
# CELLULE 9 : Test de compatibilité multi-modèles
# ========================================

print("🧪 TEST DE COMPATIBILITÉ MULTI-MODÈLES")
print("="*60)

# Test 1 : Charger avec le même modèle (normal)
print("\n📌 Test 1 : Chargement avec modèle identique")
vs_test1 = JobVectorStore(model_name="all-mpnet-base-v2")
vs_test1.load(str(index_path), str(metadata_path))
print(f"✅ Test 1 réussi\n")

# Test 2 : Charger avec un modèle différent (auto-correction)
print("📌 Test 2 : Chargement avec modèle différent (auto-correction)")
vs_test2 = JobVectorStore(model_name="all-MiniLM-L6-v2")  # Modèle différent
vs_test2.load(str(index_path), str(metadata_path))  # Doit auto-corriger
print(f"✅ Test 2 réussi : modèle auto-corrigé en {vs_test2.model_name}\n")

# Test 3 : Fonction load_vector_store (sans spécifier le modèle)
print("📌 Test 3 : Chargement via fonction utilitaire")
vs_test3 = load_vector_store(str(index_path), str(metadata_path))
print(f"✅ Test 3 réussi : {vs_test3.model_name} chargé automatiquement\n")

print("🎉 Tous les tests de compatibilité passés !")

🧪 TEST DE COMPATIBILITÉ MULTI-MODÈLES

📌 Test 1 : Chargement avec modèle identique


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Index chargé : 25 offres
📌 Modèle final : all-mpnet-base-v2 (768 dimensions)
✅ Test 1 réussi

📌 Test 2 : Chargement avec modèle différent (auto-correction)


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

...

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Modèle rechargé : all-mpnet-base-v2
✅ Index chargé : 25 offres
📌 Modèle final : all-mpnet-base-v2 (768 dimensions)
✅ Test 2 réussi : modèle auto-corrigé en all-mpnet-base-v2

📌 Test 3 : Chargement via fonction utilitaire


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Index chargé : 25 offres
📌 Modèle final : all-mpnet-base-v2 (768 dimensions)
✅ Test 3 réussi : all-mpnet-base-v2 chargé automatiquement

🎉 Tous les tests de compatibilité passés !
